In [1]:
import pandas as pd
import torch
import lightning as L

from torch.utils.data import DataLoader
from lightning.pytorch.callbacks import ModelCheckpoint

from model.modeling_demolta import DeMOLTaConfig
from trainer import LitMOLA, get_mola_dataloader

In [2]:
torch.set_float32_matmul_precision('medium')

In [3]:
BATCH_SIZE = 4
MAX_STEP = 100000000
TEXT_MODEL_NAME = 'facebook/galactica-125m'

In [4]:
smiles_to_filter = pd.read_csv('./data/test.csv')['SMILES'].tolist()

In [5]:
train_dataloader = get_mola_dataloader(
    df_path='./preproc/pretrain.csv',
    ignore_smiles=smiles_to_filter,
    tokenizer_name=TEXT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

Using pad_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [6]:
val_dataloader = get_mola_dataloader(
    df_path='./preproc/pretrain_val.csv',
    ignore_smiles=smiles_to_filter,
    tokenizer_name=TEXT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

Using pad_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [7]:
demolta_config = DeMOLTaConfig(
    num_layers=12,
    hidden_dim=384,
    ff_dim=1536,
    num_heads=6,
)

In [8]:
lit_model = LitMOLA(
    demolta_config=demolta_config,
    text_model_name=TEXT_MODEL_NAME,
)

In [9]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./checkpoint/',
    filename='mola-pretrain-{step}-{val_loss:.2f}',
    save_top_k=3,
)

In [10]:
trainer = L.Trainer(
    accelerator='gpu',
    precision='16-mixed',
    max_steps=MAX_STEP,
    callbacks=[checkpoint_callback],
    gradient_clip_val=1.0,
    val_check_interval=10000,
    limit_val_batches=1000,
)


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(lit_model, train_dataloader, val_dataloader)

Missing logger folder: c:\Users\dust\Documents\Dacon\med\demolta\lightning_logs
c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\dust\Documents\Dacon\med\demolta\checkpoint exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params
--------------------------------
0 | model | MOLLA | 48.5 M
--------------------------------
48.5 M    Trainable params
0         Non-trainable params
48.5 M    Total params
193.984   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
